In [ ]:
import cv2 as cv

In [ ]:
#open camera 
import cv2 as cv
vid = cv.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
  
    # Display the resulting frame
    cv.imshow('frame', frame)
      
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv.destroyAllWindows()

In [ ]:
#turns video to gray
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   
    # Display the resulting frame
    cv2.imshow('frame',frame)
    cv2.imshow('gray',gray)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
#changing resolution
import cv2

cap = cv2.VideoCapture(0)

def make_1080p():
    cap.set(3, 1920)
    cap.set(4, 1080)

def make_720p():
    cap.set(3, 1280)
    cap.set(4, 720)

def make_480p():
    cap.set(3, 640)
    cap.set(4, 480)

def change_res(width, height):
    cap.set(3, width)
    cap.set(4, height)

make_480p()

    
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
   
    # Display the resulting frame
    cv2.imshow('frame',frame)
   
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Python program to explain os.listdir() method
	
# importing os module
import os

# Get the list of all files and directories
# in the root directory
path = "/"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

# print the list
print(dir_list)


In [2]:
#image directories training model
import os
import cv2
import numpy as np
from PIL import Image
import pickle

BASE_DIR = cwd = os.getcwd()
img_dir = os.path.join(BASE_DIR, "images")

face_cascade = cv2.CascadeClassifier('C:/Users/Sahil Khotkar/Downloads/ML_OpenCv_project/ML OpenCv project/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()


current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root,dirs,files in os.walk(img_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(" ", "-").lower()
#             print(label, path)
            if label in label_ids:
                pass
            else:
                label_ids[label] = current_id
                current_id += 1

            id = label_ids[label]
#             print(label_ids)
#             y_labels.append(lable) #some number
#             x_train.append(path) #verify this image, turn into NUMPY array,GRAY
            pil_image = Image.open(path).convert("L") #grayscale
            size = (250,250)
            final_image = pil_image.resize(size, Image.ANTIALIAS)


            image_array = np.array(final_image, "uint8")
#             print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)

            for (x,y,w,h) in faces:
                roi = image_array[y: y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id)


# print(y_labels)
print(x_train)

with open("labels.pickle", 'wb') as f:
    pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainer.yml")
    

[]


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-i1s8y2i1\opencv_contrib\modules\face\src\lbph_faces.cpp:362: error: (-210:Unsupported format or combination of formats) Empty training data was given. You'll need more than one sample to learn a model. in function 'cv::face::LBPH::train'


In [1]:
#Face detection
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('C:/Users/Sahil Khotkar/Downloads/ML_OpenCv_project/ML OpenCv project/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

labels = {"person_name": 1}
with open("labels.pickle", 'rb') as f:
    og_labels =   pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}


cap = cv2.VideoCapture(0)

while(True):

    ret,frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for(x,y,w,h) in faces:
#         print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w] #(ycordinate_start,ycordinate_end)
        roicolor = frame[y:y+h, x:x+w]

        id, conf = recognizer.predict(roi_gray)
        if conf>= 45 and conf<= 85:
#             print(id)
#             print(labels[id_])


            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id]
            color = (255,255,255)
            stroke = 2
            final_conf = "{:.2f}".format(conf)          #to limit confidence value upto 2 decimal places
            cv2.putText(frame, name, (x,y-5), font, 1, color, stroke, cv2.LINE_AA)
            cv2.putText(frame, final_conf, (x,y-35), font, 1, color, stroke, cv2.LINE_AA)


        img_item = "6.png"
        cv2.imwrite(img_item, roi_gray)

        #Rectangle around frame
        color = (255,0,0) #BGR not RGB (Blue color)
        stroke = 2
        end_cord_x = x+w
        end_cord_y = y+h
        cv2.rectangle(frame, (x,y), (end_cord_x,end_cord_y), color, stroke)


    cv2.imshow('frame',frame)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

name = labels[id]
print(name)

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-i1s8y2i1\opencv_contrib\modules\face\src\facerec.cpp:61: error: (-2:Unspecified error) File can't be opened for reading! in function 'cv::face::FaceRecognizer::read'


In [1]:
#QRCODE RECOGNITION
import cv2
import numpy as np
from pyzbar.pyzbar import decode


cap =cv2.VideoCapture(0)
cap.set(3, 640)  #sets width of frame
cap.set(4, 480)  #sets height of frame

while True:

    success , img =cap.read()

    for barcode in decode(img):
        mydata = barcode.data.decode('utf-8')
#         print(mydata)
        pts =  np.array([barcode.polygon], np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(img, [pts] ,True, (225 ,0, 0), 4)


    cv2.imshow("frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print(mydata)

cap.release()
cv2.destroyAllWindows()

NameError: name 'mydata' is not defined

In [4]:
from tkinter import *
from PIL import ImageTk,Image
if(mydata==name):
    msg="Successfully Validated!"
    msgColor = "#3efa2d"
else:
    msg = "Invalid. Try Again :("
    msgColor = "#000000"
window =Tk()
window.geometry("500x200")
window.title("Validator")

#displaying message 
message = Message(window, text= msg)
message.config(font=("Consolas", 15, "bold"))
message.config(aspect =550)
message.config(fg=msgColor) #foreground color 
#message.config(relief =SUNKEN) #frame styling
message.config(padx = 50)     #padding on x axis
message.config(pady = 70)     #padding on y axis

message.pack()
#message.place(x=80,y=10)
window.mainloop()


SyntaxError: invalid syntax (<ipython-input-1-f6ffb92c6441>, line 1)